# 🌟 Kepler 系外行星偵測 - 完整訓練與比較 (2025)

## 📊 本 Notebook 功能

✅ **訓練 3 個模型**：Genesis CNN、XGBoost、Random Forest  
✅ **自動下載數據**：從 Kaggle 直接下載 5,657 筆光變曲線  
✅ **GPU 加速**：TensorFlow + XGBoost GPU  
✅ **完整比較**：所有模型指標、圖表、PDF 報告  
✅ **一鍵下載**：所有結果打包下載  

---

## ⚡ 預計執行時間

| GPU | 總時間 |
|-----|--------|
| **A100** | 5-8 分鐘 |
| **L4** | 8-12 分鐘 |
| **T4** | 12-15 分鐘 |

---

## 🚀 使用方式

1. **啟用 GPU**：Runtime → Change runtime type → GPU → A100 (或 L4/T4)
2. **執行所有 cells**：Runtime → Run all (或逐個執行)
3. **上傳 Kaggle API key**：執行時會提示
4. **等待完成**：自動下載結果

---

**更新日期**：2025-10-05  
**TensorFlow 版本**：2.18+ (Colab 預裝)  
**數據集**：Kaggle Kepler Labelled Time Series (5,657 samples)


---
# 🔧 步驟 1：環境檢查與設置

In [ ]:
# ========== 檢查 GPU 可用性 ==========
import subprocess
import sys

print("=" * 80)
print("GPU 檢查")
print("=" * 80)

try:
    subprocess.run(['nvidia-smi'], check=True)
except:
    print("⚠️  警告：未偵測到 GPU！")
    print("請確認：Runtime → Change runtime type → Hardware accelerator → GPU")

print("\n" + "=" * 80)
print("TensorFlow GPU 檢查")
print("=" * 80)

import tensorflow as tf
print(f"TensorFlow 版本: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
print(f"GPU 裝置: {gpus}")

if gpus:
    print(f"\n✅ GPU 可用！裝置: {gpus[0].name}")
    # 設定記憶體成長
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("\n❌ GPU 不可用，將使用 CPU（訓練會較慢）")

print("=" * 80)

In [ ]:
# ========== 安裝必要套件 ==========
print("\n正在安裝必要套件...")
!pip install -q imbalanced-learn xgboost reportlab seaborn
print("✅ 套件安裝完成！")

---
# 📥 步驟 2：下載 Kaggle 數據集

In [ ]:
# ========== 設置 Kaggle API ==========
from google.colab import files
import os

print("=" * 80)
print("Kaggle API 設置")
print("=" * 80)
print()
print("📝 請依照以下步驟：")
print("  1. 訪問 https://www.kaggle.com/settings")
print("  2. 點擊 'Create New API Token'")
print("  3. 下載 kaggle.json 檔案")
print("  4. 在下方上傳該檔案")
print()
print("請上傳 kaggle.json：")

uploaded = files.upload()

# 設置 Kaggle 配置
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 驗證
if os.path.exists(os.path.expanduser('~/.kaggle/kaggle.json')):
    print("\n✅ Kaggle API 設置成功！")
else:
    print("\n❌ Kaggle API 設置失敗，請重新上傳")

print("=" * 80)

In [ ]:
# ========== 下載 Kaggle 數據集 ==========
print("\n=" * 80)
print("下載 Kepler 數據集")
print("=" * 80)
print()

# 下載並解壓縮
!kaggle datasets download -d keplersmachines/kepler-labelled-time-series-data
!mkdir -p data
!unzip -q kepler-labelled-time-series-data.zip -d data/

# 檢查檔案
print("\n下載完成！檔案清單：")
!ls -lh data/

# 清理 zip 檔案
!rm kepler-labelled-time-series-data.zip

print("\n✅ 數據集準備完成！")
print("=" * 80)

---
# 🧪 步驟 3：導入套件與配置

In [ ]:
# ========== 導入所有必要套件 ==========
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import json
from pathlib import Path
from datetime import datetime

# TensorFlow / Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# ML libraries
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)
import xgboost as xgb
from imblearn.over_sampling import SMOTE

# PDF generation
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

print("✅ 所有套件導入成功！")

# ========== 設置隨機種子 ==========
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

# ========== 創建輸出目錄 ==========
REPORTS_DIR = Path('reports/kaggle_comparison')
FIGURES_DIR = REPORTS_DIR / 'figures'
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n輸出目錄已創建：{REPORTS_DIR}")

# ========== 設置繪圖風格 ==========
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("\n" + "=" * 80)
print("Kepler 系外行星偵測 - 完整模型訓練與比較")
print("=" * 80)
print(f"執行時間：{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"平台：Google Colab")
print(f"TensorFlow 版本：{tf.__version__}")
print(f"GPU：{gpus[0].name if gpus else 'CPU'}")
print("=" * 80)

---
# 📊 步驟 4：載入與預處理數據

In [ ]:
# ========== 載入數據 ==========
print("\n[1/6] 載入 Kaggle Kepler 數據集...")

train_df = pd.read_csv('data/exoTrain.csv')
test_df = pd.read_csv('data/exoTest.csv')

print(f"  訓練集形狀：{train_df.shape}")
print(f"  測試集形狀：{test_df.shape}")

# 分離特徵和標籤
X_train_raw = train_df.iloc[:, 1:].values
y_train_raw = train_df.iloc[:, 0].values
X_test = test_df.iloc[:, 1:].values
y_test = test_df.iloc[:, 0].values

# 轉換標籤：2 (行星) -> 1, 1 (非行星) -> 0
y_train_raw = (y_train_raw == 2).astype(int)
y_test = (y_test == 2).astype(int)

print(f"\n  訓練集：{X_train_raw.shape}")
print(f"    - 行星：{y_train_raw.sum()}")
print(f"    - 非行星：{len(y_train_raw) - y_train_raw.sum()}")
print(f"    - 類別不平衡：{100*y_train_raw.mean():.2f}% 行星")

print(f"\n  測試集：{X_test.shape}")
print(f"    - 行星：{y_test.sum()}")
print(f"    - 非行星：{len(y_test) - y_test.sum()}")

print("\n✅ 數據載入完成！")

In [ ]:
# ========== SMOTE 類別平衡 ==========
print("\n[2/6] 使用 SMOTE 處理類別不平衡...")

smote = SMOTE(random_state=RANDOM_STATE)
X_train, y_train = smote.fit_resample(X_train_raw, y_train_raw)

print(f"  SMOTE 後訓練集：{X_train.shape}")
print(f"    - 行星：{y_train.sum()}")
print(f"    - 非行星：{len(y_train) - y_train.sum()}")
print(f"    - 平衡度：{100*y_train.mean():.1f}% 行星")

print("\n✅ 類別平衡完成！")

---
# 🧠 步驟 5：訓練模型

In [ ]:
# ========== 模型 1: Genesis CNN ==========
print("\n" + "=" * 80)
print("[3/6] 訓練 Genesis CNN 模型（使用 GPU 加速）")
print("=" * 80)

def build_genesis_cnn():
    """建立 Genesis CNN 架構（適配 3,197 時間點）"""
    model = Sequential([
        Conv1D(64, 50, padding='same', activation='relu',
               input_shape=(X_train.shape[1], 1)),
        Conv1D(64, 50, padding='same', activation='relu'),
        MaxPooling1D(pool_size=16, strides=16),

        Conv1D(64, 12, padding='same', activation='relu'),
        Conv1D(64, 12, padding='same', activation='relu'),
        AveragePooling1D(pool_size=8),

        Dropout(0.25),
        Flatten(),

        Dense(256, activation='relu'),
        Dense(256, activation='relu'),
        Dense(2, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# 準備 CNN 輸入格式
X_train_cnn = X_train.reshape(-1, X_train.shape[1], 1)
X_test_cnn = X_test.reshape(-1, X_test.shape[1], 1)
y_train_cat = to_categorical(y_train, 2)
y_test_cat = to_categorical(y_test, 2)

# 建立模型
genesis_model = build_genesis_cnn()
print("\nGenesis CNN 架構：")
genesis_model.summary()

# 訓練
print("\n開始訓練...")
genesis_start = time.time()

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = genesis_model.fit(
    X_train_cnn, y_train_cat,
    validation_data=(X_test_cnn, y_test_cat),
    epochs=10,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

genesis_time = time.time() - genesis_start

# 評估
genesis_loss, genesis_acc = genesis_model.evaluate(
    X_test_cnn, y_test_cat, verbose=0
)

# 預測
y_pred_genesis_proba = genesis_model.predict(X_test_cnn, verbose=0)
y_pred_genesis = np.argmax(y_pred_genesis_proba, axis=1)

print(f"\n✅ Genesis CNN 訓練完成！")
print(f"  訓練時間：{genesis_time:.1f} 秒")
print(f"  測試準確率：{genesis_acc:.4f}")
print(f"  最終 epoch：{len(history.history['loss'])}")
print("=" * 80)

In [ ]:
# ========== 模型 2: XGBoost (GPU 加速) ==========
print("\n" + "=" * 80)
print("[4/6] 訓練 XGBoost 模型（使用 GPU 加速）")
print("=" * 80)

xgb_start = time.time()

# 計算類別權重
scale_pos_weight = (len(y_train) - y_train.sum()) / y_train.sum()

# 建立 XGBoost 模型（GPU 加速）
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    random_state=RANDOM_STATE,
    tree_method='gpu_hist',  # GPU 加速
    gpu_id=0,
    predictor='gpu_predictor'
)

print("\n開始訓練...")
xgb_model.fit(X_train, y_train, verbose=False)

xgb_time = time.time() - xgb_start

# 預測
y_pred_xgb = xgb_model.predict(X_test)
y_pred_xgb_proba = xgb_model.predict_proba(X_test)[:, 1]
xgb_acc = accuracy_score(y_test, y_pred_xgb)

print(f"\n✅ XGBoost 訓練完成！")
print(f"  訓練時間：{xgb_time:.1f} 秒")
print(f"  測試準確率：{xgb_acc:.4f}")
print("=" * 80)

In [ ]:
# ========== 模型 3: Random Forest ==========
print("\n" + "=" * 80)
print("[5/6] 訓練 Random Forest 模型")
print("=" * 80)

rf_start = time.time()

# 建立 Random Forest 模型
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    class_weight='balanced',
    random_state=RANDOM_STATE,
    n_jobs=-1  # 使用所有 CPU 核心
)

print("\n開始訓練...")
rf_model.fit(X_train, y_train)

rf_time = time.time() - rf_start

# 預測
y_pred_rf = rf_model.predict(X_test)
y_pred_rf_proba = rf_model.predict_proba(X_test)[:, 1]
rf_acc = accuracy_score(y_test, y_pred_rf)

print(f"\n✅ Random Forest 訓練完成！")
print(f"  訓練時間：{rf_time:.1f} 秒")
print(f"  測試準確率：{rf_acc:.4f}")
print("=" * 80)

---
# 📈 步驟 6：計算指標與生成視覺化

In [ ]:
# ========== 計算所有指標 ==========
print("\n[6/6] 計算指標並生成視覺化...")

def compute_metrics(y_true, y_pred, y_proba):
    """計算完整的評估指標"""
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'roc_auc': roc_auc_score(y_true, y_proba) if len(np.unique(y_true)) > 1 else 0.0
    }

# 計算各模型指標
genesis_metrics = compute_metrics(y_test, y_pred_genesis, y_pred_genesis_proba[:, 1])
xgb_metrics = compute_metrics(y_test, y_pred_xgb, y_pred_xgb_proba)
rf_metrics = compute_metrics(y_test, y_pred_rf, y_pred_rf_proba)

# 保存結果為 JSON
results = {
    'metadata': {
        'timestamp': datetime.now().isoformat(),
        'platform': 'Google Colab',
        'tensorflow_version': tf.__version__,
        'gpu': str(gpus[0].name) if gpus else 'CPU'
    },
    'dataset_info': {
        'train_samples': len(X_train),
        'test_samples': len(X_test),
        'features': X_train.shape[1],
        'planets_train': int(y_train.sum()),
        'planets_test': int(y_test.sum())
    },
    'genesis_cnn': {
        'metrics': genesis_metrics,
        'training_time_seconds': genesis_time
    },
    'xgboost': {
        'metrics': xgb_metrics,
        'training_time_seconds': xgb_time
    },
    'random_forest': {
        'metrics': rf_metrics,
        'training_time_seconds': rf_time
    }
}

with open(REPORTS_DIR / 'kaggle_comparison_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\n✅ 指標計算完成並保存為 JSON")

In [ ]:
# ========== 視覺化 1: 性能比較（4 項指標）==========
print("\n生成圖表 1/3：性能比較...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Model Performance Comparison - Kaggle Kepler Dataset',
             fontsize=18, fontweight='bold', y=0.995)

metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
metrics_keys = ['accuracy', 'precision', 'recall', 'f1']
models = ['Genesis CNN', 'XGBoost', 'Random Forest']
colors_bar = ['#3498db', '#e74c3c', '#2ecc71']

for idx, (metric_name, metric_key) in enumerate(zip(metrics_names, metrics_keys)):
    ax = axes[idx // 2, idx % 2]
    values = [
        genesis_metrics[metric_key],
        xgb_metrics[metric_key],
        rf_metrics[metric_key]
    ]

    bars = ax.bar(models, values, color=colors_bar, alpha=0.8, edgecolor='black', linewidth=1.5)
    ax.set_ylabel(metric_name, fontsize=13, fontweight='bold')
    ax.set_ylim(0, 1.05)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.set_axisbelow(True)

    # 添加數值標籤
    for bar, val in zip(bars, values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{val:.4f}',
                ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'performance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 圖表 1 已保存")

In [ ]:
# ========== 視覺化 2: ROC-AUC 和訓練時間 ==========
print("\n生成圖表 2/3：ROC-AUC 和訓練時間...")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('ROC-AUC Score & Training Time Comparison',
             fontsize=18, fontweight='bold')

# ROC-AUC
roc_values = [
    genesis_metrics['roc_auc'],
    xgb_metrics['roc_auc'],
    rf_metrics['roc_auc']
]
bars1 = ax1.bar(models, roc_values, color=colors_bar, alpha=0.8,
                edgecolor='black', linewidth=1.5)
ax1.set_ylabel('ROC-AUC Score', fontsize=13, fontweight='bold')
ax1.set_ylim(0, 1.05)
ax1.grid(axis='y', alpha=0.3, linestyle='--')
ax1.set_axisbelow(True)

for bar, val in zip(bars1, roc_values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.02,
             f'{val:.4f}',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

# Training Time
times = [genesis_time, xgb_time, rf_time]
bars2 = ax2.bar(models, times, color=colors_bar, alpha=0.8,
                edgecolor='black', linewidth=1.5)
ax2.set_ylabel('Training Time (seconds)', fontsize=13, fontweight='bold')
ax2.grid(axis='y', alpha=0.3, linestyle='--')
ax2.set_axisbelow(True)

for bar, val in zip(bars2, times):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + max(times)*0.02,
             f'{val:.1f}s',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'roc_time_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 圖表 2 已保存")

In [ ]:
# ========== 視覺化 3: 混淆矩陣 ==========
print("\n生成圖表 3/3：混淆矩陣...")

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Confusion Matrices', fontsize=18, fontweight='bold')

predictions = [y_pred_genesis, y_pred_xgb, y_pred_rf]
cmaps = ['Blues', 'Reds', 'Greens']

for ax, model_name, y_pred, cmap in zip(axes, models, predictions, cmaps):
    cm = confusion_matrix(y_test, y_pred)

    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, ax=ax,
                xticklabels=['Non-Planet', 'Planet'],
                yticklabels=['Non-Planet', 'Planet'],
                cbar_kws={'label': 'Count'},
                annot_kws={'fontsize': 14, 'fontweight': 'bold'})

    ax.set_title(model_name, fontsize=15, fontweight='bold', pad=10)
    ax.set_ylabel('True Label', fontsize=12, fontweight='bold')
    ax.set_xlabel('Predicted Label', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 圖表 3 已保存")
print("\n✅ 所有視覺化完成！")

---
# 📄 步驟 7：生成 PDF 報告

In [ ]:
# ========== 生成 PDF 報告 ==========
print("\n生成 PDF 報告...")

pdf_path = REPORTS_DIR / 'KAGGLE_MODEL_COMPARISON_REPORT.pdf'
doc = SimpleDocTemplate(str(pdf_path), pagesize=letter)
story = []
styles = getSampleStyleSheet()

# 標題
title = Paragraph(
    "<b>Kaggle Kepler Dataset - Model Comparison Report</b>",
    styles['Title']
)
story.append(title)
story.append(Spacer(1, 20))

# 元數據
gpu_name = str(gpus[0].name) if gpus else "CPU"

metadata = f"""
<b>Generated:</b> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}<br/>
<b>Platform:</b> Google Colab<br/>
<b>GPU:</b> {gpu_name}<br/>
<b>TensorFlow:</b> {tf.__version__}<br/>
<b>Dataset:</b> Kaggle Kepler Labelled Time Series<br/>
<b>Training Samples:</b> {len(X_train):,} (after SMOTE)<br/>
<b>Test Samples:</b> {len(X_test):,}<br/>
<b>Features:</b> {X_train.shape[1]:,} time points
"""
story.append(Paragraph(metadata, styles['Normal']))
story.append(Spacer(1, 30))

# 性能表格
story.append(Paragraph("<b>Model Performance Metrics</b>", styles['Heading2']))
story.append(Spacer(1, 12))

data = [[
    'Model', 'Accuracy', 'Precision', 'Recall',
    'F1-Score', 'ROC-AUC', 'Time (s)'
]]

data.append([
    'Genesis CNN',
    f"{genesis_metrics['accuracy']:.4f}",
    f"{genesis_metrics['precision']:.4f}",
    f"{genesis_metrics['recall']:.4f}",
    f"{genesis_metrics['f1']:.4f}",
    f"{genesis_metrics['roc_auc']:.4f}",
    f"{genesis_time:.1f}"
])

data.append([
    'XGBoost',
    f"{xgb_metrics['accuracy']:.4f}",
    f"{xgb_metrics['precision']:.4f}",
    f"{xgb_metrics['recall']:.4f}",
    f"{xgb_metrics['f1']:.4f}",
    f"{xgb_metrics['roc_auc']:.4f}",
    f"{xgb_time:.1f}"
])

data.append([
    'Random Forest',
    f"{rf_metrics['accuracy']:.4f}",
    f"{rf_metrics['precision']:.4f}",
    f"{rf_metrics['recall']:.4f}",
    f"{rf_metrics['f1']:.4f}",
    f"{rf_metrics['roc_auc']:.4f}",
    f"{rf_time:.1f}"
])

table = Table(data)
table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 11),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
    ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ('FONTSIZE', (0, 1), (-1, -1), 10)
]))
story.append(table)
story.append(Spacer(1, 30))

# 添加圖表
story.append(Paragraph("<b>Visualizations</b>", styles['Heading2']))
story.append(Spacer(1, 12))

for img_name in ['performance_comparison.png', 'roc_time_comparison.png', 'confusion_matrices.png']:
    img_path = FIGURES_DIR / img_name
    if img_path.exists():
        story.append(Image(str(img_path), width=500, height=300))
        story.append(Spacer(1, 15))

# 建立 PDF
doc.build(story)

print(f"\n✅ PDF 報告已生成：{pdf_path}")
print(f"   檔案大小：{pdf_path.stat().st_size / 1024:.1f} KB")

---
# 📊 步驟 8：結果摘要

In [ ]:
# ========== 顯示結果摘要 ==========
print("\n" + "=" * 80)
print("訓練完成 - 結果摘要")
print("=" * 80)
print()

# 表格標題
print(f"{'模型':<20} {'準確率':<12} {'F1-Score':<12} {'ROC-AUC':<12} {'時間 (秒)':<10}")
print("-" * 80)

# Genesis CNN
print(f"{'Genesis CNN':<20} {genesis_metrics['accuracy']:<12.4f} "
      f"{genesis_metrics['f1']:<12.4f} {genesis_metrics['roc_auc']:<12.4f} "
      f"{genesis_time:<10.1f}")

# XGBoost
print(f"{'XGBoost':<20} {xgb_metrics['accuracy']:<12.4f} "
      f"{xgb_metrics['f1']:<12.4f} {xgb_metrics['roc_auc']:<12.4f} "
      f"{xgb_time:<10.1f}")

# Random Forest
print(f"{'Random Forest':<20} {rf_metrics['accuracy']:<12.4f} "
      f"{rf_metrics['f1']:<12.4f} {rf_metrics['roc_auc']:<12.4f} "
      f"{rf_time:<10.1f}")

print("=" * 80)

# 找出最佳模型
best_acc_idx = np.argmax([
    genesis_metrics['accuracy'],
    xgb_metrics['accuracy'],
    rf_metrics['accuracy']
])
best_model = models[best_acc_idx]

print(f"\n🏆 最佳模型（準確率）：{best_model}")
print(f"   準確率：{[genesis_metrics, xgb_metrics, rf_metrics][best_acc_idx]['accuracy']:.4f}")
print(f"   F1-Score：{[genesis_metrics, xgb_metrics, rf_metrics][best_acc_idx]['f1']:.4f}")

# 最快模型
fastest_idx = np.argmin([genesis_time, xgb_time, rf_time])
fastest_model = models[fastest_idx]
print(f"\n⚡ 最快模型：{fastest_model}")
print(f"   訓練時間：{[genesis_time, xgb_time, rf_time][fastest_idx]:.1f} 秒")

print("\n" + "=" * 80)
print("輸出檔案")
print("=" * 80)
print(f"  📁 報告目錄：{REPORTS_DIR}")
print(f"  📊 JSON 結果：kaggle_comparison_results.json")
print(f"  📈 圖表：figures/*.png (3 張)")
print(f"  📄 PDF 報告：KAGGLE_MODEL_COMPARISON_REPORT.pdf")
print("=" * 80)

print("\n✅ 所有訓練和分析完成！")

---
# 💾 步驟 9：下載結果

In [ ]:
# ========== 打包並下載所有結果 ==========
print("\n" + "=" * 80)
print("準備下載結果")
print("=" * 80)

# 壓縮所有結果
!zip -r kaggle_results_complete.zip reports/

# 顯示壓縮檔資訊
print("\n壓縮檔內容：")
!unzip -l kaggle_results_complete.zip | tail -20

# 下載
from google.colab import files
files.download('kaggle_results_complete.zip')

print("\n✅ 結果已下載！")
print("\n解壓縮後包含：")
print("  - kaggle_comparison_results.json (完整指標)")
print("  - KAGGLE_MODEL_COMPARISON_REPORT.pdf (PDF 報告)")
print("  - figures/performance_comparison.png (性能比較圖)")
print("  - figures/roc_time_comparison.png (ROC-AUC 和時間)")
print("  - figures/confusion_matrices.png (混淆矩陣)")

print("\n" + "=" * 80)
print("🎉 全部完成！感謝使用！")
print("=" * 80)

---
# 📝 附加：詳細分類報告

In [ ]:
# ========== 可選：顯示詳細分類報告 ==========
print("\n" + "=" * 80)
print("詳細分類報告")
print("=" * 80)

print("\n【Genesis CNN】")
print(classification_report(y_test, y_pred_genesis,
                          target_names=['Non-Planet', 'Planet']))

print("\n【XGBoost】")
print(classification_report(y_test, y_pred_xgb,
                          target_names=['Non-Planet', 'Planet']))

print("\n【Random Forest】")
print(classification_report(y_test, y_pred_rf,
                          target_names=['Non-Planet', 'Planet']))

print("=" * 80)